<a href="https://colab.research.google.com/github/sagawritescode/ENDTwoPointOPhase1/blob/main/Sagar_Sentiment_Analysis_using_LSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [1]:
import random
import torch, torchtext.legacy
from torchtext.legacy import data

## Source of data: PyTreebank

In [3]:
!pip install pytreebank

  Created wheel for pytreebank: filename=pytreebank-0.2.7-cp37-none-any.whl size=37070 sha256=af713eaa89228e69968ec04ddc33e1d93b99256b27046973c43ea3f69e81a4c3
  Stored in directory: /root/.cache/pip/wheels/e0/b6/91/e9edcdbf464f623628d5c3aa9de28888c726e270b9a29f2368
Successfully built pytreebank


In [4]:
import pytreebank
dataset = pytreebank.load_sst()

In [5]:
Review = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [6]:
fields = [('reviews', Review),('labels',Label)]

In [7]:
# Use list to collect data as it is more efficient for appending
totaldata = []

# Pytreebank has three different datasets. Combining all of them 
for databatch in [dataset["train"], dataset["dev"], dataset["test"]]:
  for example in databatch:
    i= 0
    for label, sentence in example.to_labeled_lines():
      if i == 0:
        totaldata.append([sentence, label])
        break

## Data Augmentation

Random Delete and Random Swap functions

In [35]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining
 
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

Backtranslate

In [36]:
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 61kB 3.5MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 1.0MB 8.5MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=cf9762e03bad23eaa47bf862c99afbd3c02004a6713f6473e55dc368ffc67203
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans


In [37]:
import random
import googletrans
 
def backtranslate(sentence):
  translator = googletrans.Translator()
 
  available_langs = list(googletrans.LANGUAGES.keys()) 
  trans_lang = random.choice(available_langs) 
 
  translations = translator.translate(sentence, dest=trans_lang).text 
 
  translations_en_random = translator.translate(translations, src=trans_lang, dest='en').text
  
  return translations_en_random

Checking class distribution

In [38]:
import collections
import matplotlib.pyplot as plt
 
# A textual way to print frequency per label 
def print_frequency(data):
  x, y = list(zip(*data))
  # frequency of labels
  label_frequency = collections.Counter(y)
  print("frequency distribution: ", label_frequency)
 
print_frequency(totaldata)

frequency distribution:  Counter({1: 3140, 3: 3111, 2: 2242, 4: 1852, 0: 1510})


In [44]:
# Define split percentage here
split_percentage = 0.75

In [45]:
def augment_data(sentence):
  n = random.randint(1,3)
  if n == 1:
    sentence = ''.join(random_deletion(list(sentence)))
  elif n == 2:
    sentence = ''.join(random_swap(list(sentence)))
  else:
    sentence = backtranslate(sentence)
  return sentence

In [46]:
def get_label_elements(trainlist, label):
   return [x for x in trainlist if x[1] == label]
 
def get_int_index(split, list_to_sample):
  return int(split*len(list_to_sample))

Finally split train-test and augment train data

In [47]:
import random
random.shuffle(totaldata)
 
split_index = get_int_index(split_percentage,totaldata)
 
trainlist = totaldata[:split_index]
testlist = totaldata[split_index:]
 
changedsent = 0
length = len(trainlist)
iterator = 0
 
print_frequency(trainlist)

frequency distribution:  Counter({1: 2351, 3: 2326, 2: 1686, 4: 1386, 0: 1142})


In [48]:
 
train_very_negative = get_label_elements(trainlist, 0)
train_negative = get_label_elements(trainlist, 1)
train_neutral = get_label_elements(trainlist, 2)
train_positive = get_label_elements(trainlist, 3)
train_very_positive = get_label_elements(trainlist, 4)

# Discrepancy: Using sample here and using shuffle and split above. We realised that, but not changing as realised that late during submission
train_list_aug = [x for label_list in [random.sample(train_very_negative, get_int_index(1, train_very_negative)), 
                              random.sample(train_negative, get_int_index(0.2, train_negative)), random.sample(train_neutral, get_int_index(0.5, train_neutral)), 
                              random.sample(train_positive, get_int_index(0.2, train_positive)), random.sample(train_very_positive, get_int_index(0.8, train_very_positive))] for x in label_list ]
 
for i in train_list_aug:
    if iterator%100 == 0:
        print("sentences changed: ", iterator)
    iterator +=1
    x,y = i
    x = augment_data(x)
    trainlist.append((x, y))
 
print_frequency(trainlist)

sentences changed:  0
sentences changed:  100
sentences changed:  200
sentences changed:  300
sentences changed:  400
sentences changed:  500
sentences changed:  600
sentences changed:  700
sentences changed:  800
sentences changed:  900
sentences changed:  1000
sentences changed:  1100
sentences changed:  1200
sentences changed:  1300
sentences changed:  1400
sentences changed:  1500
sentences changed:  1600
sentences changed:  1700
sentences changed:  1800
sentences changed:  1900
sentences changed:  2000
sentences changed:  2100
sentences changed:  2200
sentences changed:  2300
sentences changed:  2400
sentences changed:  2500
sentences changed:  2600
sentences changed:  2700
sentences changed:  2800
sentences changed:  2900
sentences changed:  3000
sentences changed:  3100
sentences changed:  3200
sentences changed:  3300
sentences changed:  3400
sentences changed:  3500
sentences changed:  3600
sentences changed:  3700
sentences changed:  3800
sentences changed:  3900
sentences ch

In [49]:
example1 = [data.Example.fromlist([trainlist[i][0], trainlist[i][1]], fields) for i in range(len(trainlist))]

In [50]:
example2 = [data.Example.fromlist([testlist[i][0], testlist[i][1]], fields) for i in range(len(testlist))]

In [51]:
SEED = 230
torch.manual_seed(SEED)

In [52]:
train = data.Dataset(example1, fields)
valid = data.Dataset(example2, fields)

In [53]:
(len(train), len(valid))

(12919, 2964)

In [54]:
Review.build_vocab(train)
Label.build_vocab(train)

In [55]:
print('Size of input vocab : ', len(Review.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Review.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

print('label freq:', list(Label.vocab.freqs.most_common(6)))

Size of input vocab :  31562
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 11381), (',', 10297), ('the', 7890), ('a', 6064), ('of', 5807), ('and', 5781), ('to', 4022), ('-', 3943), ('is', 3420), ("'s", 3368)]
Labels :  defaultdict(None, {1: 0, 3: 1, 2: 2, 4: 3, 0: 4})
label freq: [(1, 2821), (3, 2791), (2, 2529), (4, 2494), (0, 2284)]


In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [57]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 64, 
                                                            sort_key = lambda x: len(x.reviews),
                                                            sort_within_batch=True, device = device)

## Save the files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cd /content/drive/MyDrive/assign5

In [71]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Review.vocab.stoi, tokens)

In [75]:
with open('train.pkl', 'wb') as tokens: 
    pickle.dump(train, tokens)

In [ ]:
with open('valid.pkl', 'wb') as tokens: 
    pickle.dump(valid, tokens)

# Defining the models

In [58]:
import torch.nn as nn
import torch.nn.functional as F
 
class classifierLSTM(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True,
                           bidirectional = False
                           )
        
        self.dropout = nn.Dropout(dropout)
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        embedded = self.dropout(embedded)
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
 
        dense_outputs = self.dropout(dense_outputs)
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output#dense_outputs[0]

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
 
class classifierLSTM_v2(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True,
                           bidirectional = False
                           )
        
        self.dropout = nn.Dropout(dropout)
        
        self.fc1 = nn.Linear(hidden_dim, output_dim*5)

        self.fc2 = nn.Linear(output_dim*5, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        embedded = self.dropout(embedded)
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs1 = self.fc1(hidden)   

        dense_outputs1 = self.dropout(dense_outputs1)

        dense_outputs = self.fc2(dense_outputs1)
        dense_outputs = self.dropout(dense_outputs)
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output#dense_outputs[0]

### Hyperparameters

In [60]:
# Define hyperparameters
size_of_vocab = len(Review.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 2
dropout = 0.3
lr = 1e-4
# Instantiate the model
model = classifierLSTM(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [61]:
print(model)
 
#No. of trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifierLSTM(
  (embedding): Embedding(31562, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)
The model has 9,710,705 trainable parameters


In [62]:
import torch.optim as optim
 
# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
 
# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [63]:
# Our own way of printing actual vs predicted (quantity) which we later realised is called confusion matrix.
def get_pred_vs_act_per_label(pred_act):
  preds, act = list(zip(*pred_act))
  predictions = []
  actuals = []
  for i in preds:
    _, pred = torch.max(i, 1)
    for j in pred:
      predictions.append(j.item())
  for i in act:
    for j in i:
      actuals.append(j.item())
 
  assert len(predictions) == len(actuals)
 
  label_dict = {}
  
  for i in range(0, len(actuals)):
    predic = predictions[i]
    actual = actuals[i]
    if label_dict.get(actual, None) != None:
      in_dict = label_dict[actual]
      if in_dict.get(predic, None) != None:
        in_dict[predic] += 1
      else:
        in_dict[predic] = 1
    else:
      in_dict = {predic: 1}
    label_dict[actual] = in_dict
  print("label distribution: actual vs pred:", label_dict)
  return label_dict

In [64]:
import torch.nn.functional as Fun
 
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        review, review_lengths = batch.reviews   
        
        # convert to 1D tensor
        beforesqueeze = model(review, review_lengths)  
        predictions = beforesqueeze.squeeze()
        # compute the loss       
        
        loss = criterion(predictions, batch.labels)        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [65]:
def evaluate(model, iterator, criterion, preds_actual_tup):
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0
 
    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            review, review_lengths = batch.reviews
            #print(review)
            # convert to 1d tensor
            predictions = model(review, review_lengths).squeeze()
            #print(predictions)
            # compute loss and accuracy
            preds_actual_tup.append((predictions, batch.labels))
            loss = criterion(predictions, batch.labels)
 
            #print(batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [66]:
N_EPOCHS = 300
best_valid_loss = float('inf')
 
 
epochi = 0
train_acc_list = []
valid_acc_list = []
 
for epoch in range(N_EPOCHS):
 
    epochi += 1 
    
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    train_acc_list.append(train_acc)
 
    preds_actual_tup = []
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, preds_actual_tup)
    valid_acc_list.append(valid_acc)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    print(epochi)
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')
    if epochi%20 == 0:
      print(f'\t pred vs actuals: {get_pred_vs_act_per_label(preds_actual_tup)}')

1
	Train Loss: 1.607 | Train Acc: 21.84%
	 Val. Loss: 1.595 |  Val. Acc: 27.93% 

2
	Train Loss: 1.604 | Train Acc: 23.13%
	 Val. Loss: 1.590 |  Val. Acc: 28.03% 

3
	Train Loss: 1.603 | Train Acc: 23.71%
	 Val. Loss: 1.589 |  Val. Acc: 28.20% 

4
	Train Loss: 1.601 | Train Acc: 24.10%
	 Val. Loss: 1.588 |  Val. Acc: 29.13% 

5
	Train Loss: 1.598 | Train Acc: 25.16%
	 Val. Loss: 1.585 |  Val. Acc: 27.66% 

6
	Train Loss: 1.595 | Train Acc: 26.07%
	 Val. Loss: 1.584 |  Val. Acc: 30.05% 

7
	Train Loss: 1.591 | Train Acc: 26.88%
	 Val. Loss: 1.581 |  Val. Acc: 30.18% 

8
	Train Loss: 1.586 | Train Acc: 27.61%
	 Val. Loss: 1.579 |  Val. Acc: 29.98% 

9
	Train Loss: 1.577 | Train Acc: 29.77%
	 Val. Loss: 1.576 |  Val. Acc: 29.81% 

10
	Train Loss: 1.568 | Train Acc: 30.89%
	 Val. Loss: 1.569 |  Val. Acc: 29.43% 

11
	Train Loss: 1.558 | Train Acc: 30.88%
	 Val. Loss: 1.562 |  Val. Acc: 30.71% 

12
	Train Loss: 1.548 | Train Acc: 33.64%
	 Val. Loss: 1.560 |  Val. Acc: 31.13% 

13
	Train Los

In [ ]:
print(train_acc_list)
print(valid_acc_list)

[0.21647315346487037, 0.22305585600469754, 0.22028208364090418, 0.22451784431117855, 0.2238757210235073, 0.22926304778551945, 0.22856710123144872, 0.2364052611671082, 0.23527372223601495, 0.24240842543236196, 0.24392423700524246, 0.244272210214236, 0.2545812304161455, 0.2541093386743711, 0.2607971842158331, 0.2582286910651481, 0.26447218221072194, 0.2726232676201215, 0.27459344716921247, 0.2738449290462825, 0.2876587159013095, 0.28631313587432583, 0.28707917776281977, 0.29130742811176874, 0.2943040034542345, 0.3018843126079263, 0.30072523633094683, 0.31263142878606437, 0.31345004210733385, 0.32096150762414277, 0.3155917047637783, 0.3168759513391207, 0.31602980053588137, 0.322950462636338, 0.32902247053847467, 0.3285330549630945, 0.3326699311874773, 0.3337413882964278, 0.3294956139777893, 0.336398752312682, 0.33931521874040227, 0.3390022931305785, 0.34568137665317483, 0.3442556877114457, 0.34998973605295297, 0.35023882481605495, 0.3543869662502585, 0.3530576584273822, 0.3595852860032695

In [ ]:
max(valid_acc_list)

0.4015957446808511

### Model diagnosis and Functions for showing 

In [ ]:
preds, act = list(zip(*preds_actual_tup))
zero_preds = 0
same_probability = 0
for i in preds:
  for j in i:
    maxv, pred = torch.max(j, dim = 0)
    minv, pred = torch.min(j, dim = 0)
    if pred == 0:
      zero_preds += 1
      if maxv - minv < 0.1:
        same_probability += 1
      
zero_preds, same_probability

(123, 0)

In [ ]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_review(review):
    
    categories = {0: "Very Negative", 1: "Negative", 2: "Neutral", 3:"Positive", 4:"Very Positive"}
    
    # tokenize the review 
    tokenized = [tok.text for tok in nlp.tokenizer(review)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()], pred.item()

In [ ]:
def classify_and_print(test_list):
  for i in test_list:
    x,y = i
    cat  = {0: "Very Negative", 1: "Negative", 2: "Neutral", 3:"Positive", 4:"Very Positive"}
    actual_label = cat[y]
    predicted_label_str, predicted_label = classify_review(x) 
    print("sentence: ", x)
    print("actual_label: ", actual_label, "predicted_label: ", predicted_label_str)
    print("")

In [ ]:
testdata_to_test = random.sample(testlist, 25)
classify_and_print(testdata_to_test)

sentence:  Possibly not since Grumpy Old Men have I heard a film so solidly connect with one demographic while striking out with another .
actual_label:  Neutral predicted_label:  Very Negative

sentence:  Burns never really harnesses to full effect the energetic cast .
actual_label:  Negative predicted_label:  Negative

sentence:  ( Screenwriter ) Pimental took the Farrelly Brothers comedy and feminized it , but it is a rather poor imitation .
actual_label:  Neutral predicted_label:  Negative

sentence:  Unless you 're a fanatic , the best advice is : ` Scooby ' do n't .
actual_label:  Negative predicted_label:  Very Positive

sentence:  The film takes the materials of human tragedy and dresses them in lovely costumes , Southern California locations and star power .
actual_label:  Neutral predicted_label:  Negative

sentence:  It is philosophy , illustrated through everyday events .
actual_label:  Positive predicted_label:  Very Negative

sentence:  Daughter From Danang reveals that e

In [ ]:
count = 0
false_pos_list = []
for i in testlist:
  x,y = i
  _, predicted_label = classify_review(x) 
  if predicted_label in [3,4] and y in [0,1]:
    false_pos_list.append(i)
    count += 1
    if count == 10:
      break

print(false_pos_list)

[["The movie is silly beyond comprehension , and even if it were n't silly , it would still be beyond comprehension .", 0], ['One of those decades-spanning historical epics that strives to be intimate and socially encompassing but fails to do justice to either effort in three hours of screen time .', 1], ['Alas , getting there is not even half the interest .', 0], ["It 's quite diverting nonsense .", 1], ["... Liotta is put in an impossible spot because his character 's deceptions ultimately undo him and the believability of the entire scenario .", 1], ['After the first 10 minutes , which is worth seeing , the movie sinks into an abyss of clichés , depression and bad alternative music .', 1], ['Witless and utterly pointless .', 1], ["The filmmakers juggle and juxtapose three story lines but fail to come up with one cogent point , unless it 's that life stinks , especially for sensitive married women who really love other women .", 1], ['I hated every minute of it .', 0], ['Why he was g

In [ ]:
classify_and_print(false_pos_list)

sentence:  The movie is silly beyond comprehension , and even if it were n't silly , it would still be beyond comprehension .
actual_label:  Very Negative predicted_label:  Positive

sentence:  One of those decades-spanning historical epics that strives to be intimate and socially encompassing but fails to do justice to either effort in three hours of screen time .
actual_label:  Negative predicted_label:  Positive

sentence:  Alas , getting there is not even half the interest .
actual_label:  Very Negative predicted_label:  Positive

sentence:  It 's quite diverting nonsense .
actual_label:  Negative predicted_label:  Positive

sentence:  ... Liotta is put in an impossible spot because his character 's deceptions ultimately undo him and the believability of the entire scenario .
actual_label:  Negative predicted_label:  Positive

sentence:  After the first 10 minutes , which is worth seeing , the movie sinks into an abyss of clichés , depression and bad alternative music .
actual_labe

In [ ]:

for i in testdata_to_test:
  x,y = i
  cat  = {0: "Very Negative", 1: "Negative", 2: "Neutral", 3:"Positive", 4:"Very Positive"}
  actual_label = cat[y]
  predicted_label = classify_review(x) 
  print("sentence: ", x)
  print("actual_label: ", actual_label, "predicted_label: ", predicted_label)
  print("")

In [ ]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [ ]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [ ]:
preds2 = []
for i in preds:
  _, predictions = torch.max(i, 1)
  #print(predictions)
  #break
  for j in predictions:
    preds2.append(j)

In [ ]:
dic = get_pred_vs_act_per_label(preds_actual_tup)

label distribution: actual vs pred: {3: {1: 194, 4: 32, 3: 105, 0: 194, 2: 33}, 2: {2: 202, 1: 203, 3: 23, 0: 32, 4: 6}, 1: {3: 84, 1: 426, 0: 127, 4: 26, 2: 143}, 0: {0: 354, 3: 121, 4: 80, 1: 159, 2: 27}, 4: {1: 72, 3: 41, 0: 201, 4: 59, 2: 20}}


In [ ]:
final_su = 0
final_co = 0
for keys in dic:
  key = dic[keys]
  su = 0
  predicted = 0
  for keys2 in key:
     su += key[keys2]
     if keys2 == keys:
       predicted = key[keys2]
  print(keys, su, predicted)
  final_su += su
  final_co += predicted
print(final_su, final_co)
print(final_co/final_su)

3 558 105
2 466 202
1 806 426
0 741 354
4 393 59
2964 1146
0.3866396761133603
